In [ ]:
import sys
import traceback

def show_exception(e):
    print("❌ Exception caught:")
    traceback.print_exception(type(e), e, e.__traceback__, file=sys.stdout)


In [ ]:
import os
from rdflib import Graph, Namespace
import ipywidgets as widgets

shacl_output = widgets.Output()

# === SHACL-Regeln anwenden auf hochgeladene Datei ===
def apply_shacl_rules(instance_file_path: str) -> Graph:
    SH = Namespace("http://www.w3.org/ns/shacl#")
    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    shapes_dir = "Shapes"  # relative Pfadangabe im Repo

    shape_files = [
        "_generic-shapes.ttl", "A1-shapes.ttl", "A2-shapes.ttl", "A3-shapes.ttl",
        "F1-shapes.ttl", "F2-shapes.ttl", "F3-shapes.ttl", "F4-shapes.ttl",
        "I1-shapes.ttl", "I2-shapes.ttl", "I3-shapes.ttl", "I4-shapes.ttl",
        "I5-shapes.ttl", "I6-shapes.ttl", "I7-shapes.ttl", "O1-shapes.ttl",
        "O2-shapes.ttl", "O3-shapes.ttl", "O4-shapes.ttl", "T1-shapes.ttl",
        "T2-shapes.ttl", "T3-shapes.ttl", "T4-shapes.ttl", "T5-shapes.ttl",
        "T6-shapes.ttl", "T7-shapes.ttl", "T8-shapes.ttl", "T9-shapes.ttl",
        "T10-shapes.ttl", "T11-shapes.ttl", "T12-shapes.ttl", "T13-shapes.ttl",
        "T14-shapes.ttl", "T15-shapes.ttl", "T16-shapes.ttl", "T17-shapes.ttl",
        "T18-shapes.ttl", "T19-shapes.ttl", "T20-shapes.ttl", "T21-shapes.ttl",
        "T22-shapes.ttl", "T23-shapes.ttl", "Y1-shapes.ttl", "Y2-shapes.ttl", "Y4-shapes.ttl"
    ]

    g_instance = Graph()
    g_instance.parse(instance_file_path, format="turtle")

    triples_before = len(g_instance)

    for shape_file in shape_files:
        shape_path = os.path.join(shapes_dir, shape_file)
        g_shape = Graph()

        if not os.path.exists(shape_path):
            with shacl_output:
                print(f"⚠️ Shape file not found: {shape_path}")
            continue

        try:
            g_shape.parse(shape_path, format="turtle")
            with shacl_output:
                print(f"✅ Loaded SHACL shape file: {shape_file}")
        except Exception as e:
            with shacl_output:
                print(f"❌ Error loading {shape_file}: {e}")
            continue

        for rule in g_shape.subjects(predicate=SH.rule, object=None):
            for _, _, construct_query in g_shape.triples((rule, SH.construct, None)):
                query = str(construct_query)
                try:
                    g_instance.update(query)
                except Exception as e:
                    with shacl_output:
                        print(f"❌ Error executing rule from {shape_file}: {e}")

    triples_after = len(g_instance)
    with shacl_output:
        print("✅ All rules applied.")
        print(f"📊 Triples before: {triples_before}")
        print(f"📈 Triples after: {triples_after}")
        print(f"➕ Added: {triples_after - triples_before} triples")

    return g_instance


In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from rdflib import Graph

# === Globale Variablen ===
instance_file_path = None
output = widgets.Output()
result_graph = None



# === Auswahlfeld: Beispiel oder Upload ===
option_selector = widgets.ToggleButtons(
    options=[("Use example file", "example"), ("Upload your own", "upload")],
    description="Select input:",
    style={'description_width': 'initial'}
)

# === Upload-Widget (immer sichtbar) ===
upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

# === "Continue"-Button ===
continue_button = widgets.Button(description="Continue", button_style='primary')

# === Auswahlhandler ===
def on_option_change(change):
    output.clear_output()
    with output:
        if change['new'] == 'example':
            global instance_file_path
            instance_file_path = "Instance_Files/swemls-instances.ttl"
            print(f"📁 Example file selected:\n→ {instance_file_path}")
        elif change['new'] == 'upload':
            print("📤 Please upload a TTL file using the field below.")

option_selector.observe(on_option_change, names='value')

# === Upload-Handler ===
def on_upload(change):
    global instance_file_path
    if upload_widget.value:
        uploaded = upload_widget.value[0]
        file_name = uploaded['name']
        instance_file_path = file_name
        with open(file_name, 'wb') as f:
            f.write(uploaded['content'])
        output.clear_output()
        with output:
            print(f"✅ File uploaded and saved as:\n→ {file_name}")

upload_widget.observe(on_upload, names='value')

# === Continue-Button-Handler ===
def on_continue(b):
    global result_graph
    output.clear_output()
    if not instance_file_path:
        with output:
            print("⚠️ No file selected or uploaded.")
        return

    if not os.path.exists(instance_file_path):
        with output:
            print(f"❌ File not found on disk:\n→ {instance_file_path}")
        return

    try:
        if instance_file_path.endswith(".ttl") and not instance_file_path.startswith("Instance_Files/"):
            with output:
                print("🔍 SHACL rules will be applied to uploaded file.")
            result_graph = apply_shacl_rules(instance_file_path)
        else:
            g = Graph()
            g.parse(instance_file_path, format="turtle")
            result_graph = g

        with output:
            print(f"✅ RDF file successfully loaded!")
            print(f"📄 Triples in graph: {len(result_graph)}")
            print(f"🔗 Using file:\n→ {instance_file_path}")

    except Exception as e:
        with output:
            print(f"❌ Error parsing TTL file:\n→ {e}")

    # ⬇️ SHACL-Ausgabe sichtbar machen
    display(shacl_output)

continue_button.on_click(on_continue)

# === Anzeige aller Elemente ===
display(option_selector, upload_widget, continue_button, output)


In [ ]:
import ipywidgets as widgets
from rdflib.namespace import RDF
from IPython.display import display

# Outputs
query_interface_output = widgets.Output()
selection_output = widgets.Output()

# SPARQL Query-Feld
query_input = widgets.Textarea(
    value="""
PREFIX swemls: <https://w3id.org/semsys/ns/swemls#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?system
WHERE {
  ?system a swemls:System .
  ?system swemls:hasCorrespondingPattern ?pattern .
  FILTER(STRENDS(STR(?pattern), "O1"))
}
""",
    placeholder='Enter your SPARQL query here...',
    description='SPARQL Query:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

# Buttons
run_query_button = widgets.Button(description="Run Query", button_style='primary')
confirm_button = widgets.Button(description="Confirm selection", button_style='success')

# Dropdown für Systeme
system_selector = widgets.Dropdown(
    options=[],
    description="Select system:",
    layout=widgets.Layout(width='50%')
)

# Globales Ergebnis
matched_systems = []

# Query-Handler
def on_query_run(b):
    global matched_systems
    query_interface_output.clear_output()
    selection_output.clear_output()
    system_selector.options = []

    if 'result_graph' not in globals() or result_graph is None:
        with query_interface_output:
            print("⚠️ RDF graph not loaded.")
        return

    try:
        results = result_graph.query(query_input.value)
    except Exception as e:
        with query_interface_output:
            print(f"❌ Query error: {e}")
        return

    matched_systems = []
    for row in results:
        uri = str(row.system)
        sys_id = uri.split("/")[-1]
        matched_systems.append((sys_id, uri))

    if not matched_systems:
        with query_interface_output:
            print("⚠️ No matching systems found.")
        return

    system_selector.options = [(sys_id, uri) for sys_id, uri in matched_systems]

    with query_interface_output:
        print(f"✅ Found {len(matched_systems)} matching system(s):")
        for i, (sys_id, _) in enumerate(matched_systems):
            print(f" {i+1}: {sys_id}")

# Auswahl-Handler
def on_confirm_selection(b):
    global selected_system_id, selected_system_uri
    selected_label = system_selector.label
    selected_uri = system_selector.value
    selected_system_id = selected_label
    selected_system_uri = selected_uri
    with selection_output:
        selection_output.clear_output()
        print(f"✅ You selected: {selected_system_id}")
        print(f"🔗 URI: {selected_system_uri}")

# Event-Bindings
run_query_button.on_click(on_query_run)
confirm_button.on_click(on_confirm_selection)

# Anzeigen
display(
    widgets.VBox([
        query_input,
        run_query_button,
        query_interface_output,
        widgets.HBox([system_selector, confirm_button]),
        selection_output
    ])
)


In [ ]:
import json
from rdflib import Graph, Namespace, URIRef, RDF
import ipywidgets as widgets
from IPython.display import display

json_output = widgets.Output()

def extract_and_export_selected_system():
    global selected_system_id, selected_system_uri, result_graph

    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    OPMW = Namespace("http://www.opmw.org/ontology#")
    RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

    rdf_type = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type")
    ai_system_type = SWEMLS.System
    documentation_type = SWEMLS.Documentation
    paper_type = SWEMLS.Paper
    ml_component_type = SWEMLS.MachineLearningComponent
    kr_component_type = SWEMLS.KnowledgeRepresentationComponent
    data_type = SWEMLS.Data
    semantic_web_resource_type = SWEMLS.SemanticWebResource
    has_documentation = SWEMLS.hasDocumentation
    reports_on = SWEMLS.reports
    rdfs_label = RDFS.label

    g = result_graph

    # 1. System und zugehörige Paper finden
    ai_systems = [
        system for system in g.subjects(RDF.type, ai_system_type)
        if str(system).split("/")[-1] == selected_system_id
    ]

    system_to_paper = {}
    for paper in g.subjects(predicate=rdf_type, object=paper_type):
        for reported_system in g.objects(subject=paper, predicate=reports_on):
            system_id = str(reported_system).split("/")[-1]
            paper_id = str(paper).split("/")[-1]
            paper_metadata = {"id": paper_id, "metadata": {}}
            for pred, obj in g.predicate_objects(subject=paper):
                pred_name = pred.split("#")[-1] if "#" in pred else pred.split("/")[-1]
                paper_metadata["metadata"].setdefault(pred_name, []).append(str(obj))
            system_to_paper.setdefault(system_id, []).append(paper_metadata)

    # 2. System extrahieren
    for system in ai_systems:
        instance_data = {
            "id": str(system).split("/")[-1],
            "type": "System",
            "metadata": {},
            "relationships": {},
            "documentation": {},
            "papers": [],
            "steps": [],
            "variables": {}
        }

        for pred, obj in g.predicate_objects(subject=system):
            pred_name = pred.split("#")[-1] if "#" in pred else pred.split("/")[-1]
            if isinstance(obj, URIRef):
                instance_data["relationships"].setdefault(pred_name, []).append(str(obj).split("/")[-1])
            else:
                instance_data["metadata"][pred_name] = str(obj)

        # 3. Dokumentation extrahieren
        if "hasDocumentation" in instance_data["relationships"]:
            for doc_id in instance_data["relationships"]["hasDocumentation"]:
                doc_uri = URIRef(f"http://semantic-systems.net/swemls/{doc_id}")
                if (doc_uri, rdf_type, documentation_type) in g:
                    instance_data["documentation"]["id"] = doc_id
                    for doc_pred, doc_obj in g.predicate_objects(subject=doc_uri):
                        doc_pred_name = doc_pred.split("#")[-1] if "#" in doc_pred else doc_pred.split("/")[-1]
                        instance_data["documentation"][doc_pred_name] = str(doc_obj)

        # 4. Paper hinzufügen
        if selected_system_id in system_to_paper:
            instance_data["papers"] = system_to_paper[selected_system_id]

        # 5. Schritte (ML/KR) extrahieren
        for i in range(1, 11):
            for step_relation in [f"hasStepML{i}", f"hasStepKR{i}"]:
                if step_relation in instance_data["relationships"]:
                    for step_id in instance_data["relationships"][step_relation]:
                        step_uri = URIRef(f"http://semantic-systems.net/swemls/{step_id}")
                        step_type = "Unknown"
                        if (step_uri, rdf_type, ml_component_type) in g:
                            step_type = "Machine Learning"
                        elif (step_uri, rdf_type, kr_component_type) in g:
                            step_type = "Knowledge Representation"
                        step_data = {"id": step_id, "type": step_type, "metadata": {}}
                        for step_pred, step_obj in g.predicate_objects(subject=step_uri):
                            step_pred_name = step_pred.split("#")[-1] if "#" in step_pred else step_pred.split("/")[-1]
                            step_data["metadata"].setdefault(step_pred_name, []).append(str(step_obj))
                        instance_data["steps"].append(step_data)

        # 6. Variablen extrahieren
        for i in range(1, 11):
            for var_relation in [f"hasVariableData{i}", f"hasVariableSW{i}"]:
                if var_relation in instance_data["relationships"]:
                    for var_id in instance_data["relationships"][var_relation]:
                        var_uri = URIRef(f"http://semantic-systems.net/swemls/{var_id}")
                        label = None
                        for _, _, label_value in g.triples((var_uri, rdfs_label, None)):
                            label = str(label_value)
                            break
                        instance_data["variables"][var_relation] = {"id": var_id, "label": label}

        # 7. Speichern
        json_filename = f"{selected_system_id}.json"
        with open(json_filename, "w") as f:
            json.dump(instance_data, f, indent=4)

        with json_output:
            json_output.clear_output()
            print(f"✅ JSON successfully exported as: {json_filename}")
            print(json.dumps(instance_data, indent=2))  # Ausgabe für Überprüfung

# === Button zum Starten der Extraktion ===
extract_button = widgets.Button(description="Extract JSON", button_style="success")
extract_button.on_click(lambda b: extract_and_export_selected_system())

display(extract_button, json_output)


In [ ]:
import json
from IPython.display import display
import ipywidgets as widgets

pattern_output = widgets.Output()

# === Funktion zur ID-Deduplizierung ===
def deduplicate_ids(variables):
    seen_ids = {}
    updated_variables = {}

    for key, var in variables.items():
        original_id = var["id"]
        label = var.get("label", "")

        if original_id in seen_ids:
            seen_ids[original_id] += 1
            new_id = f"{original_id}_{seen_ids[original_id]}"
        else:
            seen_ids[original_id] = 1
            new_id = original_id

        updated_variables[key] = {
            "id": new_id,
            "label": label
        }

    return updated_variables

# === JSON-Datei einlesen und Variablen setzen ===
def load_instance_json(json_filename):
    global instance_data, pattern

    try:
        with open(json_filename, "r") as json_file:
            extracted_data = json.load(json_file)
            instance_data = extracted_data

        # System-Label und Pattern-URI extrahieren
        system_label = instance_data.get("metadata", {}).get("label", "no label")
        raw_pattern_uri = instance_data.get("relationships", {}).get("hasCorrespondingPattern", [None])[0]

        # Sicher extrahieren – falls kein Pattern vorhanden, als "no pattern" setzen
        if raw_pattern_uri:
            pattern = raw_pattern_uri.split(".")[-1] if "Pattern." in raw_pattern_uri else raw_pattern_uri.split("/")[-1]
        else:
            pattern = "no pattern"

        pattern_info = f"🧩 Pattern detected: {pattern}" if pattern != "no pattern" else "⚠️ No pattern detected"

        # IDs deduplizieren
        instance_data["variables"] = deduplicate_ids(instance_data.get("variables", {}))

        with pattern_output:
            pattern_output.clear_output()
            print(f"✅ Loaded instance: {system_label}")
            print(f"📎 raw_pattern_uri: {raw_pattern_uri}")
            print(f"🧩 pattern: {pattern}")
            print(pattern_info)

        # Danach weitere Pattern-Entscheidung anzeigen
        handle_pattern_selection()

    except FileNotFoundError:
        with pattern_output:
            pattern_output.clear_output()
            print("❌ JSON file not found! Please extract an instance first.")
    except Exception as e:
        with pattern_output:
            pattern_output.clear_output()
            print(f"❌ Error loading JSON:\n→ {e}")



    except FileNotFoundError:
        with pattern_output:
            pattern_output.clear_output()
            print("❌ JSON file not found! Please extract an instance first.")
    except Exception as e:
        with pattern_output:
            pattern_output.clear_output()
            print(f"❌ Error loading JSON:\n→ {e}")

# === Button zur Ausführung ===
load_button = widgets.Button(description="Load Extracted JSON", button_style="primary")
def on_load_click(b):
    try:
        json_file = f"{selected_system_id}.json"
        load_instance_json(json_file)
    except NameError:
        with pattern_output:
            pattern_output.clear_output()
            print("❌ No system selected. Please extract a system first.")

load_button.on_click(on_load_click)


display(load_button, pattern_output)


In [ ]:
import os
import json
import re
from rdflib import Graph, Namespace, RDF
import ipywidgets as widgets
from IPython.display import display, clear_output

# === Globals for reuse ===
pattern_structure = None
pattern_choice = None
pattern_graph = None
pattern = None

# === Variables from previous steps assumed ===
# instance_data (dict), pattern (str or "no pattern")

# === Output containers ===
pattern_output = widgets.Output()
pattern_upload_output = widgets.Output()

download_button = widgets.Button(
    description="⬇️ Download Template XML",
    button_style="info",
    icon="download"
)
download_output = widgets.Output()


pattern_continue_button = widgets.Button(
    description="Continue with uploaded pattern",
    button_style="success"
)

# === Upload widget (only shown when needed) ===
pattern_upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload Pattern File'
)

# === Pattern decision: Template or Generate ===
pattern_choice_selector = widgets.ToggleButtons(
    options=[
        ("Use template", "template"),
        ("Generate automatically", "generate")
    ],
    description="Select Pattern Option:",
    style={'description_width': 'initial'}
)
# ✅ Template generieren und Download-Button anzeigen
def generate_template_from_instance():
    print("🛠 generate_template_from_instance() aufgerufen")
    global pattern, instance_data, pattern_structure

    try:
        print(f"🔍 Aktuelles Pattern: {pattern}")
        print(f"📦 instance_data vorhanden: {'instance_data' in globals()}")
        print(f"📦 pattern_structure vorhanden: {'pattern_structure' in globals()}")

        if not pattern or pattern in ("no pattern", "None", None):
            print("⚠️ Kein gültiges Pattern gesetzt. Abbruch.")
            return

        template_path = f"Templates/{pattern}.xml"
        print(f"📄 Template-Pfad: {template_path}")

        if not os.path.exists(template_path):
            print(f"❌ Template-Datei nicht gefunden: {template_path}")
            return

        with open(template_path, "r", encoding="utf-8") as f:
            xml_template = f.read()
            print("✅ Template-Datei geladen")

        updated_xml = replace_pattern_placeholders(xml_template, instance_data, pattern_structure)
        print("✅ XML erfolgreich ersetzt")

        # Mapping-Name z. B. nach Instanz-ID setzen
        instance_id = instance_data.get("id", "unknown")
        output_path = f"Updated_{instance_id}_Workflow.xml"
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(updated_xml)

        print(f"✅ Template erfolgreich generiert: {output_path}")

        with download_output:
            download_output.clear_output()
            file_dl = FileDownload(
                data=updated_xml,
                filename=output_path,
                description="⬇️ Download XML",
                button_style="success"
            )
            display(file_dl)

    except Exception as e:
        import traceback
        print("❌ Fehler beim Generieren des Templates:")
        traceback.print_exception(type(e), e, e.__traceback__)

def handle_pattern_selection():
    global pattern, pattern_choice, pattern_structure

    pattern_output.clear_output()
    pattern_upload_output.clear_output()

    try:
        with pattern_output:
            print(f"🔍 DEBUG pattern: {pattern}")

            if pattern is None or pattern == "no pattern":
                print("📂 No pattern linked to system. Please upload a pattern TTL file.")
                display(pattern_upload_widget)
                display(pattern_continue_button)  # ⬅️ Neuer Button wird angezeigt
                display(pattern_upload_output)
            else:
                print(f"🧩 Pattern detected: {pattern}")
                print("How would you like to proceed?")
                display(pattern_choice_selector)
    except Exception as e:
        show_exception(e)
        

def on_pattern_continue(b):
    global pattern_structure

    with pattern_output:
        pattern_output.clear_output()
        print("🟢 [DEBUG] Button clicked!")

        if not pattern_upload_widget.value:
            print("🔴 [DEBUG] No file in upload_widget.value")
            print("⚠️ No file uploaded. Please upload a pattern file first.")
            return

        upload_items = list(pattern_upload_widget.value)
        print(f"🟡 [DEBUG] Upload items found: {len(upload_items)}")

        if not upload_items:
            print("❌ Upload seems empty.")
            return

        upload_info = upload_items[0]
        file_name = upload_info['name']
        file_content = upload_info['content']

        print(f"🟢 [DEBUG] File name = {file_name}")
        print(f"🟢 [DEBUG] File content size = {len(file_content)} bytes")

        try:
            with open(file_name, "wb") as f:
                f.write(file_content)
            print("🟢 [DEBUG] File written to disk.")

            pattern_structure = extract_pattern_structure_from_file(file_name)
            print("✅ Pattern structure parsed:")
            print(json.dumps(pattern_structure, indent=4))

        except Exception as e:
            print(f"❌ Error reading pattern file: {file_name}")
            print(str(e))

# Am Ende registrieren:
pattern_continue_button.on_click(on_pattern_continue)


# === Extract logic from TTL file ===
def extract_pattern_structure_from_file(file_path):
    g = Graph()
    g.parse(file_path, format="turtle")

    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    OPMW = Namespace("http://www.opmw.org/ontology/")
    RES = Namespace("http://semantic-systems.net/swemls/")

    def clean_uri(uri):
        label = uri.split("/")[-1].split("#")[-1]
        label = re.sub(r'Pattern\.[A-Za-z0-9]+\.', '', label)
        label = re.sub(r'^[A-Za-z0-9]+\.', '', label)
        return label

    structure = {"steps": {}, "variables": {}}

    for step in g.subjects(RDF.type, SWEMLS.WorkflowTemplateProcessML):
        step_label = clean_uri(str(step))
        inputs = [clean_uri(str(var)) for var in g.objects(step, OPMW["uses"])]
        structure["steps"][step_label] = {"type": "ML", "uses": inputs, "outputs": []}

    for step in g.subjects(RDF.type, SWEMLS.WorkflowTemplateProcessKR):
        step_label = clean_uri(str(step))
        inputs = [clean_uri(str(var)) for var in g.objects(step, OPMW["uses"])]
        structure["steps"][step_label] = {"type": "KR", "uses": inputs, "outputs": []}

    for step in g.subjects(RDF.type, RES.WorkflowTemplateProcessML):
        step_label = clean_uri(str(step))
        inputs = [clean_uri(str(var)) for var in g.objects(step, OPMW["uses"])]
        structure["steps"][step_label] = {"type": "ML", "uses": inputs, "outputs": []}

    for step in g.subjects(RDF.type, RES.WorkflowTemplateProcessKR):
        step_label = clean_uri(str(step))
        inputs = [clean_uri(str(var)) for var in g.objects(step, OPMW["uses"])]
        structure["steps"][step_label] = {"type": "KR", "uses": inputs, "outputs": []}

    for var in g.subjects(RDF.type, SWEMLS.TemplateArtifactData):
        var_label = clean_uri(str(var))
        generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]
        for gen in generated_by:
            if gen in structure["steps"]:
                structure["steps"][gen]["outputs"].append(var_label)
        structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

    for var in g.subjects(RDF.type, RES.TemplateArtifactData):
        var_label = clean_uri(str(var))
        generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]
        for gen in generated_by:
            if gen in structure["steps"]:
                structure["steps"][gen]["outputs"].append(var_label)
        structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

    for var in g.subjects(RDF.type, SWEMLS.TemplateArtifactSW):
        var_label = clean_uri(str(var))
        generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]
        for gen in generated_by:
            if gen in structure["steps"]:
                structure["steps"][gen]["outputs"].append(var_label)
        structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

    for var in g.subjects(RDF.type, RES.TemplateArtifactSW):
        var_label = clean_uri(str(var))
        generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]
        for gen in generated_by:
            if gen in structure["steps"]:
                structure["steps"][gen]["outputs"].append(var_label)
        structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

    return structure

# === Trigger on pattern upload ===
def on_pattern_upload(change):
    global pattern_structure

    if not pattern_upload_widget.value:
        return

    uploaded = next(iter(pattern_upload_widget.value.items()))  # (filename, fileinfo)
    file_name = uploaded[0]
    file_content = uploaded[1]['content']

    with open(file_name, "wb") as f:
        f.write(file_content)

    with pattern_output:
        clear_output()
        print(f"✅ Pattern file uploaded: {file_name}")

    try:
        pattern_structure = extract_pattern_structure_from_file(file_name)
        with pattern_output:
            print("✅ Extracted pattern structure:")
            print(json.dumps(pattern_structure, indent=4))
    except Exception as e:
        with pattern_output:
            print(f"❌ Error parsing pattern file: {e}")




pattern_upload_widget.observe(on_pattern_upload, names='value')



def on_pattern_choice(change):
    global pattern_choice, pattern_structure
    pattern_choice = change['new']
    file_path = f"Patterns/{pattern}-pattern.ttl"

    pattern_output.clear_output()

    try:
        pattern_structure = extract_pattern_structure_from_file(file_path)

        with pattern_output:
            print(f"✅ Pattern '{pattern}' loaded from: {file_path}")
            print(f"🧩 Pattern mode selected: {pattern_choice}")
            print("📊 Pattern structure:")
            print(json.dumps(pattern_structure, indent=4))

        # ⬅️ Hier Template erzeugen, wenn "Use template" gewählt wurde
        if pattern_choice == "template":
            generate_template_from_instance()

    except Exception as e:
        with pattern_output:
            print(f"❌ Error reading pattern file: {file_path}")
            print(f"{e}")


pattern_choice_selector.observe(on_pattern_choice, names='value')




# Danach Anzeige und Auswahl starten
main_pattern_box = widgets.VBox([
    pattern_output,
    pattern_upload_output,
    download_output  # 🆕 Zeigt den Download-Link nach Template-Erstellung
])


# Button-Handler aktivieren ⬇️
pattern_continue_button.on_click(on_pattern_continue)

# Danach Anzeige und Auswahl starten
handle_pattern_selection()
display(main_pattern_box)


In [ ]:
import xml.etree.ElementTree as ET
import re
import os
from IPython.display import display, HTML
import ipywidgets as widgets

# === Download-Ausgabe Widget ===
download_output = widgets.Output()

# === Funktion: Label-Short extrahieren ===
def extract_label_short(step):
    labels = step.get("metadata", {}).get("label", [])
    if labels:
        match = re.search(r"\((.*?)\)", labels[0])
        if match:
            return match.group(1)
    return step["id"].split(".")[-1]

# === Platzhalter im XML durch konkrete Werte ersetzen ===
def replace_pattern_placeholders(xml_string, instance_data, pattern_structure):
    with download_output:
        print("🔧 replace_pattern_placeholders() aufgerufen")
        print(f"🔧 XML Länge: {len(xml_string)}")
        print(f"🔧 Instance data keys: {list(instance_data.keys())}")
        print(f"🔧 Pattern structure keys: {list(pattern_structure.keys())}")

    tree = ET.ElementTree(ET.fromstring(xml_string))
    root = tree.getroot()

    replacements = {}

    with download_output:
        print("🔁 Schritte werden ersetzt...")
    for step in instance_data.get("steps", []):
        step_key = step["id"].split(".")[-1]
        step_label = extract_label_short(step)
        replacements[step_key] = step_label

    with download_output:
        print("🔁 Variablen werden ersetzt...")
    for pattern_var in pattern_structure.get("variables", {}):
        for var_key, var_data in instance_data.get("variables", {}).items():
            if pattern_var.lower() in var_key.lower():
                inst_id = var_data["id"].replace("Resource.", "").replace("Custom.", "")
                replacements[pattern_var] = inst_id
                break

    with download_output:
        print("\n🧩 Mapping for replacements:")
        for k, v in replacements.items():
            print(f"  {k} → {v}")

    for element in root.iter("mxCell"):
        if 'value' in element.attrib:
            value = element.attrib['value']
            for placeholder, real_value in replacements.items():
                if placeholder in value:
                    replacement = real_value.replace("_", " ") if real_value.strip() else "Missing"
                    style = element.attrib.get("style", "")
                    length = len(replacement)
                    if length > 24:
                        style += ";fontSize=5"
                    elif length > 20:
                        style += ";fontSize=7"
                    elif length > 17:
                        style += ";fontSize=9"
                    element.attrib['style'] = style
                    element.attrib['value'] = value.replace(placeholder, replacement)

    return ET.tostring(root, encoding='utf8', method='xml').decode()

# ✅ Template generieren und HTML-Link anzeigen
def generate_template_from_instance():
    global pattern, instance_data, pattern_structure

    with download_output:
        download_output.clear_output()
        try:
            print("🛠 generate_template_from_instance() aufgerufen")
            print(f"🔍 Aktuelles Pattern: {pattern}")
            print(f"📦 instance_data vorhanden: {'instance_data' in globals()}")
            print(f"📦 pattern_structure vorhanden: {'pattern_structure' in globals()}")

            if not pattern or pattern in ("no pattern", "None", None):
                print("⚠️ Kein gültiges Pattern gesetzt. Abbruch.")
                return

            template_path = f"Templates/{pattern}.xml"
            print(f"📄 Template-Pfad: {template_path}")

            if not os.path.exists(template_path):
                print(f"❌ Template-Datei nicht gefunden: {template_path}")
                return

            with open(template_path, "r", encoding="utf-8") as f:
                xml_template = f.read()
                print(f"✅ Template-Datei geladen mit Länge {len(xml_template)}")

            updated_xml = replace_pattern_placeholders(xml_template, instance_data, pattern_structure)
            print("✅ XML erfolgreich ersetzt")

            instance_id = instance_data.get("id", "unknown")
            output_filename = f"Updated_{instance_id}_Workflow.xml"
            with open(output_filename, "w", encoding="utf-8") as f:
                f.write(updated_xml)

            print(f"✅ Template erfolgreich generiert: {output_filename}")

            # HTML-Download-Link anzeigen
            html_link = f'<a href="files/{output_filename}" download target="_blank">⬇️ Click here to download the XML file</a>'
            display(HTML(html_link))

        except Exception as e:
            import traceback
            print("❌ Fehler beim Generieren des Templates:")
            traceback.print_exception(type(e), e, e.__traceback__)

# 🆕 Button zum manuellen Triggern der Template-Generierung
generate_button = widgets.Button(
    description="Generate Template XML",
    button_style="warning",
    icon="cogs"
)

def on_generate_click(b):
    with download_output:
        print("🟡 generate_button clicked")
        if "instance_data" in globals() and "pattern_structure" in globals():
            print("✅ Instanzdaten und Patternstruktur vorhanden – Generierung wird gestartet")
            generate_template_from_instance()
        else:
            print("⚠️ Instanzdaten oder Patternstruktur fehlen")
            print("⚠️ Bitte zuerst Instanz laden und Pattern extrahieren.")

generate_button.on_click(on_generate_click)

# 🆕 Anzeige des Buttons und Ausgabecontainers
display(generate_button, download_output)